In [7]:
import numpy as np
import tensorflow as tf


model.h5


In [2]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.optimizers import SGD
import numpy as np
import pandas as pd



Using TensorFlow backend.


In [4]:
train_data=pd.read_csv("dataset/Training.csv")
test_data=pd.read_csv("dataset/Testing.csv")

list(train_data.columns)

['itching',
 'skin_rash',
 'nodal_skin_eruptions',
 'continuous_sneezing',
 'shivering',
 'chills',
 'joint_pain',
 'stomach_pain',
 'acidity',
 'ulcers_on_tongue',
 'muscle_wasting',
 'vomiting',
 'burning_micturition',
 'spotting_ urination',
 'fatigue',
 'weight_gain',
 'anxiety',
 'cold_hands_and_feets',
 'mood_swings',
 'weight_loss',
 'restlessness',
 'lethargy',
 'patches_in_throat',
 'irregular_sugar_level',
 'cough',
 'high_fever',
 'sunken_eyes',
 'breathlessness',
 'sweating',
 'dehydration',
 'indigestion',
 'headache',
 'yellowish_skin',
 'dark_urine',
 'nausea',
 'loss_of_appetite',
 'pain_behind_the_eyes',
 'back_pain',
 'constipation',
 'abdominal_pain',
 'diarrhoea',
 'mild_fever',
 'yellow_urine',
 'yellowing_of_eyes',
 'acute_liver_failure',
 'fluid_overload',
 'swelling_of_stomach',
 'swelled_lymph_nodes',
 'malaise',
 'blurred_and_distorted_vision',
 'phlegm',
 'throat_irritation',
 'redness_of_eyes',
 'sinus_pressure',
 'runny_nose',
 'congestion',
 'chest_pain',


In [135]:
y_train=np.array(train_data['prognosis'])
x_train=np.array(train_data.drop(columns=['prognosis','Unnamed: 133']))


y_test=np.array(test_data['prognosis'])
x_test=np.array(test_data.drop(columns=['prognosis']))

array([0, 1])

In [137]:
#np.unique([1, 1, 2, 2, 3, 3])
values=np.unique(y_train)
values

array(['(vertigo) Paroymsal  Positional Vertigo', 'AIDS', 'Acne',
       'Alcoholic hepatitis', 'Allergy', 'Arthritis', 'Bronchial Asthma',
       'Cervical spondylosis', 'Chicken pox', 'Chronic cholestasis',
       'Common Cold', 'Dengue', 'Diabetes ',
       'Dimorphic hemmorhoids(piles)', 'Drug Reaction',
       'Fungal infection', 'GERD', 'Gastroenteritis', 'Heart attack',
       'Hepatitis B', 'Hepatitis C', 'Hepatitis D', 'Hepatitis E',
       'Hypertension ', 'Hyperthyroidism', 'Hypoglycemia',
       'Hypothyroidism', 'Impetigo', 'Jaundice', 'Malaria', 'Migraine',
       'Osteoarthristis', 'Paralysis (brain hemorrhage)',
       'Peptic ulcer diseae', 'Pneumonia', 'Psoriasis', 'Tuberculosis',
       'Typhoid', 'Urinary tract infection', 'Varicose veins',
       'hepatitis A'], dtype=object)

In [138]:


Y_train=np.zeros(shape=(len(y_train),len(values)))
k=0
for x in y_train:
     for i in range(len(values)):
            if x==values[i]:
                tmp=list(np.zeros(41))
                tmp[i]=1
                Y_train[k]=tmp
                k+=1


In [139]:
Y_train[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0.])

In [140]:
np.unique(Y_train)


array([0., 1.])

In [221]:
model = Sequential()
model.add(Dense(4920, input_dim=132))
model.add(Activation('tanh'))
model.add(Dense(41))
model.add(Activation('sigmoid'))

sgd = SGD(lr=0.1)
model.compile(loss='binary_crossentropy',metrics=['accuracy'], optimizer=sgd)

model.fit(x_train, Y_train, batch_size=1, nb_epoch=3)


/home/wajahat/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  # Remove the CWD from sys.path while we load stuff.


Epoch 1/3
4920/4920 [==============================] - 53s 11ms/step - loss: 0.0765 - accuracy: 0.9797
Epoch 2/3
4920/4920 [==============================] - 56s 11ms/step - loss: 0.0170 - accuracy: 0.9973
Epoch 3/3
4920/4920 [==============================] - 53s 11ms/step - loss: 0.0084 - accuracy: 0.9997


In [142]:
#np.unique([1, 1, 2, 2, 3, 3])
test_values=np.unique(y_test)
test_values

Y_test=np.zeros(shape=(len(y_test),len(test_values)))
k=0
for x in y_test:
     for i in range(len(test_values)):
            if x==test_values[i]:
                tmp=list(np.zeros(41))
                tmp[i]=1
                Y_test[k]=tmp
                k+=1


In [143]:
pre=model.predict_proba(x_test)

In [215]:
acc=0;
for i in range(len(pre)):
    if pre[i].argmax() == Y_test[i].argmax() :
        if pre[i].max() >= 0.90:
            acc+=1

acc=acc/len(pre)
acc

0.8571428571428571

In [217]:
model.save("model/model.h5")

In [219]:
MODEL_PATH="model/model.h5"
TFLITE_MODEL_PATH="model/kmodel.tflite"

model = tf.keras.models.load_model(MODEL_PATH)
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converted_tflite_model = converter.convert()
open(TFLITE_MODEL_PATH, "wb").write(converted_tflite_model)

3425880

In [220]:
# Load TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path=TFLITE_MODEL_PATH)

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

interpreter.allocate_tensors()

#interpreter.pre

# input details
print("INPUT TYPES AND SHAP OF THE MODEL \n")
print(input_details)
# output details
print("OUTPUT TYPES AND SHAP OF THE MODEL \n")
print(output_details)

INPUT TYPES AND SHAP OF THE MODEL 

[{'name': 'dense_19_input', 'index': 1, 'shape': array([  1, 132], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0)}]
OUTPUT TYPES AND SHAP OF THE MODEL 

[{'name': 'Identity', 'index': 0, 'shape': array([ 1, 41], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0)}]
